In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-09 14:22:04--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-09 14:22:04 (31.1 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
# Load the documents, create the index

import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)


In [3]:
# Define the search Query, specifying the course

def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [7]:
# prompt with OpenAI gpt3.5-turbo Template


def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [5]:
# Combining all to RAG Query

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [6]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [8]:
llm('what is a test')

" A test, in its most basic form, refers to an assessment procedure designed to evaluate the knowledge and abilities of individuals. It can take many forms including written exams, practical tests, oral interviews, computer-based simulations, among others.\n\nThe purpose behind testing varies based on context but often includes: \n1. Gauging one's understanding in a specific subject area (academic test)\n2. Identifying strengths and weaknessin individuals to guide further learning or development programs (psychological assessments, aptitude tests, etc.)  \n3. Certifying competence for professional fields such as medicine, law, engineering - where licensing examinations are undertaken \n4. Gauging job performance in workplaces through appraisal systems and promotions processes (workplace evaluations)   \n5. Determining the effectiveness of an educational program or teaching methodology by assessing student learning outcomes  \n6. Assessment tools to identify specific development needs f

In [ ]:
# to run it in docker

- run 